In [1]:
from functions import *
import pandas as pd
from pathlib import Path

In [2]:
# Define the parameters up top
OPENAI_MODEL = 'text-davinci-003'
TRANSFORMERS_MODEL = 'sentence-transformers/paraphrase-mpnet-base-v2'
SIMILARITY_THRESHOLD = 0.75
NUMBER_OF_TRIES = 5
PRE_CONTEXT = "Answer the following question in a concise manner."
POST_CONTEXT  = "Generate the response in JSON format with key 'answer'."


In [4]:
import pandas as pd
from pathlib import Path


# Initialize the models and set the API key
set_openai_api_key()
openai_llm = initialize_openai_llm(OPENAI_MODEL)
similarity_model = initialize_similarity_model(TRANSFORMERS_MODEL)
similar_json_behavior = SimilarJSON(similarity_model, SIMILARITY_THRESHOLD)

# Define a function to apply the evaluation to each row
def apply_evaluation(row):
    return evaluate_row(row, openai_llm, similar_json_behavior, PRE_CONTEXT, POST_CONTEXT)

# Read the CSV file into a DataFrame
csv_file_path = Path('../data/prompt_target_answer_pairs.csv')
df = pd.read_csv(csv_file_path).head(2)

# Apply the evaluation function to each row and collect the results into a list
all_results = df.apply(apply_evaluation, axis=1).tolist()

# Now all_results is a list of evaluation results
# Each element corresponds to the evaluation of one prompt/best_answer pair


Applying Perturbations: 100%|██████████| 6/6 [00:03<00:00,  1.73it/s]


In [6]:
final_df = aggregate_results(all_results)
final_df

{'original_prompt': 'What is the powerhouse of the cell?',
 'pre_context': 'Answer the following question in a concise manner.',
 'post_context': "Generate the response in JSON format with key 'answer'.",
 'perturbed_prompts': ['What is the powerhouse of the cell?',
  "The cell's powerhouse is known as the mitochondria.",
  'The mitochondria is often referred to as the powerhouse of the cell.',
  "The cell's energy generator is called the mitochondria.",
  'The mitochondria serves as the powerhouse of the cell.',
  'The powerhouse of the cell is the mitochondria.'],
 'perturbed_generations': ['\n\n{"answer": "The powerhouse of the cell is the mitochondria."}',
  '\n\n{"answer":"The cell\'s powerhouse is known as the mitochondria."}',
  '\n\n{"answer": "The mitochondria is known as the powerhouse of the cell because it is the organelle responsible for producing the majority of the energy used by the cell."}',
  '\n\n{"answer": "The cell\'s energy generator is called the mitochondria."}'